# A Faster Horse

Just a quick script to check the stats of all my equines.

In [1]:
import json
from os import environ
from pathlib import Path
from typing import Any, Collection, Dict

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
save_folder = Path(environ["SAVE_PATH"])

In [3]:
def get_name(entity: Dict[str, Any]) -> str:
    """Return the name (or identifier) of an entity

    Parameters
    ----------
    entity: dict
        The entity of interest

    Returns
    -------
    str
        The name and ID (or just the ID) of the entity
    """
    identifier = entity["id"]

    if "CustomName" not in entity.keys():
        return str(identifier)
    name = json.loads(entity["CustomName"].value)["text"]
    return f"{name} ({identifier})"

In [4]:
%%time
horse_data = []
for path in sorted((save_folder / "entities").glob("*")):
    region_data = region.RegionFile(path)
    for chunk in region_data.iter_chunks():
        for entity in chunk["Entities"]:
            if entity["id"].value not in (
                "minecraft:horse",
                "minecraft:mule",
                "minecraft:donkey",
            ):
                continue
            attributes: Dict[str, Any] = {"name": get_name(entity)}
            attributes["wild"] = "Owner" not in entity
            attributes["location"] = tuple(
                int(v.value) for i, v in enumerate(entity["Pos"]) if i != 1
            )

            for attr in entity["Attributes"]:
                if any(
                    keyword in attr["Name"].value
                    for keyword in ("armor", "knockback", "follow_range")
                ):
                    continue
                attributes[attr["Name"].value.split(":")[-1].split(".")[-1]] = attr[
                    "Base"
                ].value

            horse_data.append(attributes)

horse_dataframe = (
    pd.DataFrame(horse_data)
    .sort_values("movement_speed", ascending=False)
    .reset_index(drop=True)
)
horse_dataframe

CPU times: user 7.88 s, sys: 417 ms, total: 8.29 s
Wall time: 8.3 s


,name,wild,location,movement_speed,max_health,jump_strength
0,minecraft:horse,True,"(-1736, -4172)",0.321808,24.0,0.863070
1,minecraft:horse,True,"(585, -717)",0.317067,25.0,0.648758
2,minecraft:horse,True,"(1275, 122)",0.310454,23.0,0.700381
3,minecraft:horse,False,"(-1526, -3974)",0.308744,19.0,0.839726
4,minecraft:horse,True,"(-1604, -4410)",0.303628,18.0,0.629597
...,...,...,...,...,...,...
553,minecraft:donkey,True,"(724, 2369)",NaN,25.0,NaN
554,minecraft:donkey,True,"(1666, -8981)",NaN,22.0,NaN
555,minecraft:donkey,True,"(1665, -8978)",NaN,22.0,NaN
556,minecraft:donkey,True,"(2350, -1940)",NaN,21.0,NaN


Yowza that's a lot of wild animals! And while I definitely want to make friends with that speed demon over at -1736, -4172, let's limit this to my rides.

In [5]:
horse_dataframe.loc[~horse_dataframe.wild]

,name,wild,location,movement_speed,max_health,jump_strength
3,minecraft:horse,False,"(-1526, -3974)",0.308744,19.000000,0.839726
15,Mare-claren F1 (minecraft:horse),False,"(-1518, -3960)",0.292318,28.000000,0.602987
50,Horseracio Paganeigh (minecraft:horse),False,"(-1502, -3950)",0.272652,28.000000,0.623520
53,Lamborghi-neigh (minecraft:horse),False,"(-1510, -3982)",0.271633,27.000000,0.566055
146,minecraft:horse,False,"(-1529, -3987)",0.244324,22.000000,0.696182
181,Horsche 911 (minecraft:horse),False,"(-1509, -3948)",0.236387,27.666667,0.711469
201,minecraft:horse,False,"(-1079, -4346)",0.232124,21.000000,0.910533
205,Samuel Camino (minecraft:mule),False,"(-1506, -3961)",0.231659,28.333333,0.628128
287,Muleoo (minecraft:mule),False,"(-1501, -3968)",0.213634,24.333333,0.550304
392,Ferrar-neigh (minecraft:horse),False,"(-1500, -3949)",0.182948,29.000000,0.746220


Yeah, so Muleoo wasn't quite the hooning champ I was hoping for. But WOW I need to figure out who that stud is at Number 4*, piddling health notwithstanding.

*_Indices start at 0, remember._